# Intro to Pytorch

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn

import numpy as np

# Tensors

In [8]:
# create uninitialized tensor
x = torch.Tensor(5, 3)
# create initilalized tensor
y = torch.ones(5)
z = torch.zeros(5)

# addition
x = torch.add(y, z)
torch.add(y, z, out=x)

# modify in place - any function that modifies has a trailing _
x.add_(1)


 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

In [13]:
# moving to and from numpy - they still share the same memory
a = np.ones(5)
b = torch.from_numpy(a)
b.add_(1)
print(b, a)

c = torch.ones(3)
d = c.numpy()
d += 5
print(c, d)


 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]
 [ 2.  2.  2.  2.  2.]

 6
 6
 6
[torch.FloatTensor of size 3]
 [ 6.  6.  6.]


In [20]:
# do we have a GPU?
print(torch.cuda.is_available())
x = torch.ones(5)
y = torch.zeros(5) + 7

# move things onto the GPU (this will fail if we don't have a GPU I think)
x.cuda()
y.cuda()
print(x + y)

True

 8
 8
 8
 8
 8
[torch.FloatTensor of size 5]



# Autograd

In [56]:
# Variable "Wraps a tensor and records the operations applied to it."
# http://pytorch.org/docs/0.3.0/autograd.html#torch.autograd.Variable
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

y = x + 2
print(y)

z = y * y * 3
print(z)

out = z.mean()
print(out)

# Variable.backward "Computes the gradient of current variable w.r.t. graph leaves."
# http://pytorch.org/docs/0.3.0/autograd.html#torch.autograd.Variable.backward
out.backward()
print(x.grad) # d(out)/dx. Note that you can't do this for y or z

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]

Variable containing:
 27
[torch.FloatTensor of size 1]

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

None


In [87]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
n = 1
while y.data.norm() < 1000:
    n += 1
    y = y * 2

print(n)
print(y)
# y = 2^n * x
# dy/dx = 2^n
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)
# x.grad = 2^n scaled by the tensor we passed to y.backward. Unclear why we do this
print(x.grad)

9
Variable containing:
 378.0182
 807.5120
-659.2539
[torch.FloatTensor of size 3]

Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]



# Neural Networks

Convolutions, pooling, non-linear activation layers (relu?), linear, loss.

In [4]:
# Linear
m = nn.Linear(3, 5).cuda()
print("Weight:", m.weight)
print("Bias:", m.bias)

inp = autograd.Variable(torch.randn(2, 3)).cuda()
print("Input:", inp)

out = m(inp)
print("Output:", out)

# out = m.weight * inp + m.bias; note that weight is a 5x3 and bias is a 5x1 so that 
# (weight) 5x3 * 3x1 = 5x1 + (bias) 5x1 = 5x1 output

Weight: Parameter containing:
-0.2793 -0.5173 -0.2821
-0.3767 -0.1036  0.0083
-0.3715  0.1794  0.2099
-0.0109 -0.4441  0.1441
 0.1493  0.4208 -0.2435
[torch.cuda.FloatTensor of size 5x3 (GPU 0)]

Bias: Parameter containing:
 0.2942
 0.1223
 0.4086
-0.5541
 0.2998
[torch.cuda.FloatTensor of size 5 (GPU 0)]

Input: Variable containing:
 1.3291  1.2303  0.3277
 1.0239 -1.3926 -1.4749
[torch.cuda.FloatTensor of size 2x3 (GPU 0)]

Output: Variable containing:
-0.8058 -0.5030  0.2043 -1.0677  0.9360
 1.1446 -0.1314 -0.5313 -0.1594  0.2258
[torch.cuda.FloatTensor of size 2x5 (GPU 0)]



In [7]:
0.4131*0.4569 + 0.26*-1.0166 + 0.2009*-0.6799 + 0.5047

0.29253748000000007

In [6]:
# Conv1d(in_channels, out_channels, kernel_size)
# "channel" is ~ attribute of data
# 1 channel - grayscale image. 2 channels - height and weight of person. 3 channels - RGB image
# 1d here means that the vector is 1d (just has a length)
# This mostly just changes the number of channels, but also changes the size of the data based off how many
# full convolutions it can fit
m = nn.Conv1d(1, 3, 4)
print(m.weight.shape)
print(m.bias.shape)
# 2 variables that have 1 channel of 8 items
x = torch.randn(2, 1, 8)

inp = autograd.Variable(x)
print(inp.shape)
out = m(inp)
print(out.shape)

print(m.weight[0])
print(m.bias[0])
print(inp[0])
print(out[0])
#print(m.weight[0][:4].numpy * inp[0][:4] + bias[0])

torch.Size([3, 1, 4])
torch.Size([3])
torch.Size([2, 1, 8])
torch.Size([2, 3, 5])
Variable containing:
 0.0617  0.2367  0.2365 -0.0008
[torch.FloatTensor of size 1x4]

Variable containing:
-0.4776
[torch.FloatTensor of size 1]

Variable containing:
 2.7438  0.8303 -1.1472 -0.6355  1.9833  0.7237  0.7010  1.3415
[torch.FloatTensor of size 1x8]

Variable containing:
-0.3826 -0.8497 -0.2304  0.1231 -0.0193
-0.6302  0.0546  0.9190  1.0034  0.7299
-0.1047 -0.3683 -1.1348 -0.6199 -0.8037
[torch.FloatTensor of size 3x5]



In [8]:
0.4497*1.0882 + -0.2141*0.0440 + -0.1572*-1.5311 + -0.3080*-1.0910 + 0.4705

1.52716006